In [1]:
import cvxpy as cp
import numpy as np
import math
from queue import PriorityQueue
import networkx as nx
import matplotlib.pyplot as plt
from tabulate import tabulate
from scipy.optimize import linprog

In [4]:
class BranchAndBoundSolver:
    def __init__(self, c, A, b, maximize=True):
        """
        Initialize the Branch and Bound solver for binary integer programming.
        Parameters:
        - c: Objective coefficients (for max c'x)
        - A, b: Constraints Ax <= b
        - maximize: True for maximization, False for minimization
        """
        self.c = c
        self.A = A
        self.b = b
        self.n = len(c)  # Number of variables
        # Best solution found so far
        self.best_solution = None
        self.best_objective = float('-inf') if maximize else float('inf')
        self.maximize = maximize
        # Track nodes explored
        self.nodes_explored = 0
        # Graph for visualization
        self.graph = nx.DiGraph()
        self.node_id = 0
        # For tabular display of steps
        self.steps_table = []
        # Set of active nodes
        self.active_nodes = set()

    def is_binary_feasible(self, x):
        """Check if the solution satisfies binary constraints"""
        if x is None:
            return False
        for idx in range(self.n):
            if abs(x[idx] - 0) > 1e-6 and abs(x[idx] - 1) > 1e-6:
                return False
        return True

    def get_branching_variable(self, x):
        """Select the most fractional variable to branch on (binary case)"""
        max_fractional = -1
        branching_var = -1
        for idx in range(self.n):
            fractional_part = min(abs(x[idx] - 0), abs(x[idx] - 1))
            if fractional_part > max_fractional and fractional_part > 1e-6:
                max_fractional = fractional_part
                branching_var = idx
        return branching_var

    def solve_relaxation(self, lower_bounds, upper_bounds):
        """Solve the continuous relaxation with given bounds"""
        x = cp.Variable(self.n)
        # Set the objective - maximize c'x or minimize -c'x
        if self.maximize:
            objective = cp.Maximize(self.c @ x)
        else:
            objective = cp.Minimize(self.c @ x)
        # Basic constraints Ax <= b
        constraints = [self.A @ x <= self.b]
        # Add bounds
        for i in range(self.n):
            if lower_bounds[i] is not None:
                constraints.append(x[i] >= lower_bounds[i])
            if upper_bounds[i] is not None:
                constraints.append(x[i] <= upper_bounds[i])
        prob = cp.Problem(objective, constraints)
        try:
            objective_value = prob.solve()
            return x.value, objective_value
        except:
            return None, float('-inf') if self.maximize else float('inf')


    def add_node_to_graph(self, node_name, objective_value, x_value, parent=None, branch_var=None, branch_cond=None):
        """Add a node to the branch and bound graph"""
        self.graph.add_node(node_name, obj=objective_value, x=x_value, 
                          branch_var=branch_var, branch_cond=branch_cond)
        
        if parent is not None:
            label = f"x_{branch_var} {branch_cond}"
            self.graph.add_edge(parent, node_name, label=label)
        
        return node_name
    

    def solve(self, verbose=True):
        """Solve the problem using branch and bound"""
        # Initialize bounds for binary variables
        lower_bounds = [0] * self.n
        upper_bounds = [1] * self.n  # Binary variables are bounded between 0 and 1
        # Create a priority queue for nodes (max heap for maximization, min heap for minimization)
        node_queue = PriorityQueue()
        # Solve the root relaxation
        print("Step 1: Solving root relaxation (continuous problem)")
        x_root, obj_root = self.solve_relaxation(lower_bounds, upper_bounds)
        if x_root is None:
            print("Root problem infeasible")
            return None, float('-inf') if self.maximize else float('inf')
        # Add root node to the graph
        root_node = "S0"
        self.add_node_to_graph(root_node, obj_root, x_root)
        print(f"Root relaxation objective: {obj_root:.6f}")
        print(f"Root solution: {x_root}")
        # Initial upper bound is the root objective
        upper_bound = obj_root
        # Check if the root solution is already binary-feasible
        if self.is_binary_feasible(x_root):
            print("Root solution is binary-feasible! No need for branching.")
            self.best_solution = x_root
            self.best_objective = obj_root
            return x_root, obj_root
        # Add root node to the queue and active nodes set
        priority = -obj_root if self.maximize else obj_root
        node_queue.put((priority, self.nodes_explored, root_node, lower_bounds.copy(), upper_bounds.copy()))
        self.active_nodes.add(root_node)
        # Start branch and bound process
        node_counter = 1
        while not node_queue.empty():
            # Get the node with the highest objective (for maximization)
            priority, _, node_name, node_lower_bounds, node_upper_bounds = node_queue.get()
            self.nodes_explored += 1
            print(f"\nStep {self.nodes_explored + 1}: Exploring node {node_name}")
            # Remove from active nodes
            self.active_nodes.remove(node_name)
            # Branch on most fractional variable
            branch_var = self.get_branching_variable(self.graph.nodes[node_name]['x'])
            branch_val = self.graph.nodes[node_name]['x'][branch_var]
            print(f"  Branching on variable x_{branch_var + 1} with value {branch_val:.6f}")
            print(f"  Creating two branches: x_{branch_var + 1} = 0 and x_{branch_var + 1} = 1")
            # Process left branch (x_i = 0)
            left_node = f"S{node_counter}"
            node_counter += 1
            # Create the "x_i = 0" branch
            floor_lower_bounds = node_lower_bounds.copy()
            floor_upper_bounds = node_upper_bounds.copy()
            floor_upper_bounds[branch_var] = 0
            # Solve the relaxation for this node
            x_floor, obj_floor = self.solve_relaxation(floor_lower_bounds, floor_upper_bounds)
            # Add node to graph
            self.add_node_to_graph(left_node, obj_floor if x_floor is not None else float('-inf'), 
                                 x_floor, node_name, branch_var, f"= 0")
            # Process the floor branch
            if x_floor is None:
                print(f"  {left_node} is infeasible")
            else:
                print(f"  {left_node} relaxation objective: {obj_floor:.6f}")
                print(f"  {left_node} solution: {x_floor}")
                # Check if binary feasible and update best solution if needed
                if self.is_binary_feasible(x_floor) and ((self.maximize and obj_floor > self.best_objective) or 
                                                        (not self.maximize and obj_floor < self.best_objective)):
                    self.best_solution = x_floor.copy()
                    self.best_objective = obj_floor
                    print(f"  Found new best binary solution with objective {self.best_objective:.6f}")
                # Add to queue if not fathomed
                if ((self.maximize and obj_floor > self.best_objective) or 
                    (not self.maximize and obj_floor < self.best_objective)):
                    if not self.is_binary_feasible(x_floor):  # Only branch if not binary feasible
                        priority = -obj_floor if self.maximize else obj_floor
                        node_queue.put((priority, self.nodes_explored, left_node, 
                                       floor_lower_bounds.copy(), floor_upper_bounds.copy()))
                        self.active_nodes.add(left_node)
            # Process right branch (x_i = 1)
            right_node = f"S{node_counter}"
            node_counter += 1
            # Create the "x_i = 1" branch
            ceil_lower_bounds = node_lower_bounds.copy()
            ceil_upper_bounds = node_upper_bounds.copy()
            ceil_lower_bounds[branch_var] = 1
            # Solve the relaxation for this node
            x_ceil, obj_ceil = self.solve_relaxation(ceil_lower_bounds, ceil_upper_bounds)
            # Add node to graph
            self.add_node_to_graph(right_node, obj_ceil if x_ceil is not None else float('-inf'), 
                                  x_ceil, node_name, branch_var, f"= 1")
            # Process the ceil branch
            if x_ceil is None:
                print(f"  {right_node} is infeasible")
            else:
                print(f"  {right_node} relaxation objective: {obj_ceil:.6f}")
                print(f"  {right_node} solution: {x_ceil}")
                # Check if binary feasible and update best solution if needed
                if self.is_binary_feasible(x_ceil) and ((self.maximize and obj_ceil > self.best_objective) or 
                                                       (not self.maximize and obj_ceil < self.best_objective)):
                    self.best_solution = x_ceil.copy()
                    self.best_objective = obj_ceil
                    print(f"  Found new best binary solution with objective {self.best_objective:.6f}")
                # Add to queue if not fathomed
                if ((self.maximize and obj_ceil > self.best_objective) or 
                    (not self.maximize and obj_ceil < self.best_objective)):
                    if not self.is_binary_feasible(x_ceil):  # Only branch if not binary feasible
                        priority = -obj_ceil if self.maximize else obj_ceil
                        node_queue.put((priority, self.nodes_explored, right_node, 
                                      ceil_lower_bounds.copy(), ceil_upper_bounds.copy()))
                        self.active_nodes.add(right_node)
        print("\nBranch and bound completed!")
        print(f"Nodes explored: {self.nodes_explored}")
        if self.best_solution is not None:
            print(f"Optimal objective: {self.best_objective:.6f}")
            print(f"Optimal solution: {self.best_solution}")
        else:
            print("No feasible binary solution found")
        return self.best_solution, self.best_objective

In [5]:
if __name__ == "__main__":
    # Problem definition:
    # max 3x_1 + 2x_2
    # subject to:
    #   x_1 + x_2 <= 1
    #   x_1, x_2 ∈ {0, 1}
    c = np.array([3, 2])  # Objective coefficients
    A = np.array([[1, 1]])  # Constraint coefficients
    b = np.array([1])  # Right-hand side
    # Create and run the solver
    solver = BranchAndBoundSolver(c, A, b, maximize=True)
    solution, objective = solver.solve(verbose=True)

Step 1: Solving root relaxation (continuous problem)
Root relaxation objective: 3.000000
Root solution: [1.00000000e+00 4.59364968e-10]
Root solution is binary-feasible! No need for branching.
